# Collaboration and Competition

---

You are welcome to use this coding environment to train your agent for the project.  Follow the instructions below to get started!

### 1. Start the Environment

Run the next code cell to install a few packages.  This line will take a few minutes to run!

In [ ]:
!pip -q install ./python

The environment is already saved in the Workspace and can be accessed at the file path provided below. Now we download required files and create an environment.

In [ ]:
!rm ddpg_agent.py model.py
!curl https://raw.githubusercontent.com/VVKot/deep-reinforcement-learning/master/p3_collab-compet/ddpg_agent.py --output ddpg_agent.py
!curl https://raw.githubusercontent.com/VVKot/deep-reinforcement-learning/master/p3_collab-compet/model.py --output model.py
!curl https://raw.githubusercontent.com/VVKot/deep-reinforcement-learning/master/p2_continuous-control/workspace_utils.py --output workspace_utils.py

In [ ]:
import numpy as np
import torch
from collections import deque
from ddpg_agent import Agent
from unityagents import UnityEnvironment
from workspace_utils import active_session

env = UnityEnvironment(file_name="/data/Tennis_Linux_NoVis/Tennis")

Environments contain **_brains_** which are responsible for deciding the actions of their associated agents. Here we check for the first brain available, and set it as the default brain we will be controlling from Python.

In [ ]:
# get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]
print(brain_name)
print(brain)

### 2. Examine the State and Action Spaces

Run the code cell below to print some information about the environment.

In [ ]:
# reset the environment
env_info = env.reset(train_mode=True)[brain_name]

# number of agents 
num_agents = len(env_info.agents)
print('Number of agents:', num_agents)

# size of each action
action_size = brain.vector_action_space_size
print('Size of each action:', action_size)

# examine the state space 
states = env_info.vector_observations
state_size = states.shape[1]
print('There are {} agents. Each observes a state with length: {}'.format(states.shape[0], state_size))
print('The state for the first agent looks like:', states[0])

### 3. Take Random Actions in the Environment

In the next code cell, you will learn how to use the Python API to control the agent and receive feedback from the environment.

Note that **in this coding environment, you will not be able to watch the agents while they are training**, and you should set `train_mode=True` to restart the environment.

In [ ]:
# for i in range(5):                                         # play game for 5 episodes
#     env_info = env.reset(train_mode=False)[brain_name]     # reset the environment    
#     states = env_info.vector_observations                  # get the current state (for each agent)
#     scores = np.zeros(num_agents)                          # initialize the score (for each agent)
#     while True:
#         actions = np.random.randn(num_agents, action_size) # select an action (for each agent)
#         actions = np.clip(actions, -1, 1)                  # all actions between -1 and 1
#         env_info = env.step(actions)[brain_name]           # send all actions to tne environment
#         next_states = env_info.vector_observations         # get next state (for each agent)
#         rewards = env_info.rewards                         # get reward (for each agent)
#         dones = env_info.local_done                        # see if episode finished
#         scores += env_info.rewards                         # update the score (for each agent)
#         states = next_states                               # roll over states to next time step
#         if np.any(dones):                                  # exit loop if episode finished
#             break
#     print('Total score (averaged over agents) this episode: {}'.format(np.mean(scores)))

When finished, you can close the environment.

In [ ]:
#env.close()

### 4. It's Your Turn!

Now it's your turn to train your own agent to solve the environment!  A few **important notes**:
- When training the environment, set `train_mode=True`, so that the line for resetting the environment looks like the following:
```python
env_info = env.reset(train_mode=True)[brain_name]
```
- To structure your work, you're welcome to work directly in this Jupyter notebook, or you might like to start over with a new file!  You can see the list of files in the workspace by clicking on **_Jupyter_** in the top left corner of the notebook.
- In this coding environment, you will not be able to watch the agents while they are training.  However, **_after training the agents_**, you can download the saved model weights to watch the agents on your own machine! 

Now we can instantiate two agents to play tennis. We make the code more general since potentially the same approach might be applied for the environment with more than two agents.

In [ ]:
env_info = env.reset(train_mode=True)[brain_name]

def create_agent(states, actions):
    return Agent(states, actions, 0)

agents = [create_agent(state_size, action_size) for _ in range(num_agents)] # create two agents to play tennis

In [ ]:
from collections import deque
import torch
from workspace_utils import active_session

def ddpg(n_episodes=1500,print_every=10):
    env_info = env.reset(train_mode=True)[brain_name]
    for agent in agents: # do not forget to reset all agents
        agent.reset()
    scores_deque = deque(maxlen=100)
    scores = []
    for i_episode in range(1, n_episodes+1):
        env_info = env.reset(train_mode=True)[brain_name]
        for agent in agents:
            agent.reset() # reset every agent for each episode
        states = env_info.vector_observations
        scores = np.zeros(num_agents) # score should be represented as an array due to presence of multiple agents
        num_step = 0
        while True:
            actions = np.stack(agent.act(state) for agent, state in zip(agents, states))
            env_info = env.step(actions)[brain_name]
            next_states = env_info.vector_observations
            rewards = env_info.rewards
            dones = env_info.local_done
            for agent, s, a, r, n, d in zip(agents, states, actions, rewards, next_states, dones):
                agent.step(s, a, r, n, d) # act with every agent
            states = next_states # move all agents to next states
            score += np.array(rewards) # update scores
            num_step += 1 # move to next iteration
            if any(dones):
                break # if episode was finished due to dropping the ball - break and record score
        print('\rEpisode {}\tReward: {:.2f}'.format(i_episode, np.sum(rewards)), end="")
        scores_deque.append(np.max(score)) # get the maximum score of all agents
        scores.append(np.max(score)) # record this score
        for i, agent in enumerate(agents):
            torch.save(agent.actor_local.state_dict(), 'checkpoint_actor_{:02d}.pth'.format(i))
            torch.save(agent.critic_local.state_dict(), 'checkpoint_critic_{:02d}.pth'.format(i))
        if i_episode % print_every == 0:
            print('\rEpisode {}\tAverage Score: {:.2f}\tmax_step:{}'.format(i_episode, np.mean(scores_deque), num_step))
            if np.mean(scores_deque) >= 0.5:
                print("Score is higher than 0.5")
                break
    return scores

with active_session(): # used to avoid workspace restart due to long-running learning process
    scores = ddpg()

Now we can visualize results of the agent:

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
plt.plot(scores, 'o-')
plt.grid()
plt.title('Reward Records')
plt.xlabel('Episode')
plt.ylabel('Reward')
plt.show()

TODO Write a conclusion